In [ ]:
%matplotlib inline

In [ ]:
%config InlineBackend.figure_format = "retina"

import os
import json
import pickle
import pathlib

import h5py
import corner
import numpy as np
import pandas as pd
import lightkurve as lk
import matplotlib.pyplot as plt
from astroquery.mast import Catalogs, Observations

import pymc3 as pm
import theano.tensor as tt

import exoplanet as xo

import tess_world
tess_world.setup_notebook()

In [ ]:
toi_num = 514

tois = tess_world.get_toi_list()

# Select all of the rows in the TOI table that are associated with this target
toi = tois[tois.toi == toi_num + 0.01]
if not len(toi):
    raise RuntimeError(f"no TOI entry for {toi_num}")
toi = toi.iloc[0]
tic = toi.tid
tois = tois[tois.tid == tic].sort_values("toi")
num_toi = len(tois)

# Extract the planet periods
period_guess = np.array(tois.pl_orbper, dtype=float)

# Convert the phase to TBJD from BJD
t0_guess = np.array(tois.pl_tranmid, dtype=float) - 2457000

# Convert the depth to parts per thousand from parts per million
depth_guess = 1e-3 * np.array(tois.pl_trandep, dtype=float)

# Convert the duration to days from hours
duration_guess = np.array(tois.pl_trandurh, dtype=float) / 24.0

tois[["tid", "toi", "pl_orbper", "pl_trandep", "pl_trandurh"]]

In [ ]:
# Temporary workaround for slow MAST queries with lightkurve
observations = Observations.query_criteria(
    target_name=f"{tic}",
    radius=0.0001,
    project=["TESS"],
    obs_collection=["TESS"],
    provenance_name="SPOC",
    dataproduct_type="timeseries",
)
if not len(observations):
    raise RuntimeError("no 2-minute cadence data")
products = Observations.get_product_list(observations)
products = products[products["productSubGroupDescription"] == "LC"]
files = Observations.download_products(
    products, download_dir=tess_world.get_lightkurve_directory()
)
lcfs = lk.LightCurveCollection(
    [lk.open(file).PDCSAP_FLUX for file in files["Local Path"]]
)
lc = lcfs.stitch().remove_nans()

# # To be replaced by:
# lcfs = lk.search_lightcurvefile(f"{tic}", mission="TESS")
# print(f"Found {len(lcfs)} light curve file(s)")
# lcfs = lcfs.download_all(download_dir="./cache/lightkurve")
# lc = lcfs.PDCSAP_FLUX.stitch()
# lc = lc.remove_nans().remove_outliers()

x = np.ascontiguousarray(lc.time, dtype=np.float64)
y = np.ascontiguousarray(1e3 * (lc.flux - 1), dtype=np.float64)
yerr = np.ascontiguousarray(1e3 * lc.flux_err, dtype=np.float64)

plt.plot(x, y, "k", linewidth=0.5)
plt.xlabel("time [days]")
plt.ylabel("relative flux [ppt]")
_ = plt.title(f"TOI {toi_num}; TIC {tic}", fontsize=14)

In [ ]:
# Deal with single transits
single_transit = period_guess <= 0.0
period_guess[single_transit] = x.max() - x.min()
period_min = np.maximum(np.abs(t0_guess - x.max()), np.abs(x.min() - t0_guess))

In [ ]:
transit_mask = np.zeros_like(x, dtype=bool)
for n in range(num_toi):
    delta = max(1.5 * duration_guess[n], 0.1)
    if single_transit[n]:
        delta = 1.0
    x_fold = (x - t0_guess[n] + 0.5 * period_guess[n]) % period_guess[
        n
    ] - 0.5 * period_guess[n]
    transit_mask |= np.abs(x_fold) < delta

    plt.figure(figsize=(8, 4))
    plt.scatter(x_fold[transit_mask], y[transit_mask], c=x[transit_mask], s=3)
    plt.xlabel("time since transit [days]")
    plt.ylabel("relative flux [ppt]")
    plt.colorbar(label="time [days]")
    plt.title(f"TOI {toi_num}.{n + 1:02d}, PDC flux", fontsize=14)
    plt.xlim(-delta, delta)

x = np.ascontiguousarray(x[transit_mask])
y = np.ascontiguousarray(y[transit_mask])
yerr = np.ascontiguousarray(yerr[transit_mask])

In [ ]:
def build_model(mask=None):
    if mask is None:
        mask = np.ones_like(x, dtype=bool)

    with pm.Model() as model:
        # Stellar parameters
        mean = pm.Normal("mean", mu=0.0, sigma=10.0)
        u = xo.distributions.QuadLimbDark("u")

        # Gaussian process noise model
        sigma = pm.InverseGamma("sigma", alpha=3.0, beta=2 * np.median(yerr))
        log_S_tot = pm.Normal(
            "log_S_tot",
            mu=np.log(np.median((y[mask] - np.median(y[mask])) ** 2)),
            sigma=5.0,
        )
        log_ell = pm.Normal("log_ell", mu=np.log(1.0), sigma=5.0)
        Q = 1.0 / 3.0
        w0 = 2 * np.pi * tt.exp(-log_ell)
        S0 = tt.exp(log_S_tot + log_ell) / (2 * np.pi * Q)
        kernel = xo.gp.terms.SHOTerm(S0=S0, w0=w0, Q=Q)

        # Dealing with period, treating single transits properly
        period_params = []
        periods = []
        for n in range(num_toi):
            if single_transit[n]:
                period = pm.Pareto(
                    f"period_{n}",
                    m=period_min[n],
                    alpha=2.0 / 3,
                    testval=period_guess[n],
                )
                period_params.append(period)
            else:
                log_period = pm.Normal(
                    f"log_period_{n}", mu=np.log(period_guess[n]), sigma=1.0
                )
                period = pm.Deterministic(f"period_{n}", tt.exp(log_period))
                period_params.append(log_period)
            periods.append(period)
        period = pm.Deterministic("period", tt.stack(periods))

        # Transit parameters
        t0 = pm.Normal("t0", mu=t0_guess, sigma=1.0, shape=num_toi)
        log_dur = pm.Normal(
            "log_duration", mu=np.log(duration_guess), sigma=5.0, shape=num_toi
        )
        b = xo.distributions.UnitUniform("b", shape=num_toi)
        log_depth = pm.Normal(
            "log_depth", mu=np.log(depth_guess), sigma=5.0, shape=num_toi
        )
        depth = pm.Deterministic("transit_depth", tt.exp(log_depth))
        dur = pm.Deterministic("transit_duration", tt.exp(log_dur))

        # Compute the radius ratio from the transit depth, impact parameter, and
        # limb darkening parameters making the small-planet assumption
        u1 = u[0]
        u2 = u[1]
        mu = tt.sqrt(1 - b ** 2)
        ror = pm.Deterministic(
            "ror",
            tt.sqrt(
                1e-3
                * depth
                * (1 - u1 / 3 - u2 / 6)
                / (1 - u1 * (1 - mu) - u2 * (1 - mu) ** 2)
            ),
        )

        # Set up the orbit
        orbit = xo.orbits.KeplerianOrbit(period=period, duration=dur, t0=t0, b=b)

        # Geometric probability
        pm.Potential("geom", tt.log(orbit.dcosidb))

        # We're going to track the implied density for reasons that will become clear later
        pm.Deterministic("rho_circ", orbit.rho_star)

        # Set up the mean transit model
        star = xo.LimbDarkLightCurve(u)

        lc_model = tess_world.LightCurveModels(mean, star, orbit, ror)

        # Finally the GP observation model
        gp = xo.gp.GP(kernel, x[mask], yerr[mask] ** 2 + sigma ** 2, mean=lc_model)
        gp.marginal("obs", observed=y[mask])

        # Double check that everything looks good - we shouldn't see any NaNs!
        print(model.check_test_point())

        # Optimize the model
        map_soln = model.test_point
        map_soln = xo.optimize(map_soln, [sigma])
        map_soln = xo.optimize(map_soln, [mean, log_depth, b, log_dur])
        map_soln = xo.optimize(map_soln, [sigma, log_S_tot, log_ell])
        map_soln = xo.optimize(map_soln, [mean, u])
        map_soln = xo.optimize(map_soln, period_params)
        map_soln = xo.optimize(map_soln)

        # Save some of the key parameters
        model.map_soln = map_soln
        model.lc_model = lc_model
        model.gp = gp
        model.mask = mask
        model.x = x[mask]
        model.y = y[mask]
        model.yerr = yerr[mask]

    return model


def build_model_with_sigma_clipping(sigma=5.0, maxiter=10):
    ntot = len(x)
    mask = np.ones_like(x, dtype=bool)
    pred = np.zeros_like(y)
    for i in range(maxiter):
        print(f"Sigma clipping round {i + 1}")

        with build_model(mask) as model:
            pred[mask] = xo.eval_in_model(
                model.gp.predict() + model.lc_model(x[mask]), model.map_soln
            )
            if np.any(~mask):
                pred[~mask] = xo.eval_in_model(
                    model.gp.predict(x[~mask]) + model.lc_model(x[~mask]),
                    model.map_soln,
                )

        resid = y - pred
        rms = np.sqrt(np.median(resid ** 2))
        mask = np.abs(resid) < sigma * rms

        print(
            f"... clipping {(~mask).sum()} of {len(x)} ({100 * (~mask).sum() / len(x):.1f}%)"
        )

        if ntot == mask.sum():
            break
        ntot = mask.sum()

    return model


model = build_model_with_sigma_clipping()

In [ ]:
with model:
    gp_pred, lc_pred = xo.eval_in_model(
        [model.gp.predict(), model.lc_model.light_curves(model.x)], model.map_soln
    )

transit_mask = np.zeros_like(x, dtype=bool)
for n in range(num_toi):
    t0 = model.map_soln["t0"][n]
    period = model.map_soln["period"][n]
    x_fold = (model.x - t0 + 0.5 * period) % period - 0.5 * period

    plt.figure(figsize=(8, 4))
    plt.scatter(x_fold, model.y - gp_pred - model.map_soln["mean"], c=model.x, s=3)

    inds = np.argsort(x_fold)
    plt.plot(x_fold[inds], lc_pred[inds, n], "k")

    plt.xlabel("time since transit [days]")
    plt.ylabel("de-trended flux [ppt]")
    plt.colorbar(label="time [days]")
    plt.title(f"TOI {toi_num}.{n + 1:02d}, map model", fontsize=14)
    delta = max(1.5 * duration_guess[n], 0.1)
    if single_transit[n]:
        delta = 1.0
    plt.xlim(-delta, delta)

In [ ]:
np.random.seed(toi_num)
with model:
    trace = pm.sample(
        tune=2000,
        draws=2000,
        start=model.map_soln,
        chains=2,
        step=xo.get_dense_nuts_step(target_accept=0.9),
    )

In [ ]:
output_directory = pathlib.Path(os.environ.get("OUTPUT_DIRECTORY", "."))

# Save the model
with open(output_directory / "model.pkl", "wb") as f:
    pickle.dump(model, f, -1)

# Save the MAP solution
with open(output_directory / "map_soln.json", "w") as f:
    json.dump(model.map_soln, f, indent=2, cls=tess_world.NumpyEncoder)

# Save the summary statistics
summary = pm.summary(trace, round_to="none")
summary.to_csv(output_directory / "summary.csv")

# Save the trace
df = pm.trace_to_dataframe(trace, include_transformed=True)
stats = pd.DataFrame(
    dict((name, trace.get_sampler_stats(name)) for name in trace.stat_names)
)
with h5py.File(output_directory / "trace.h5", "w") as f:
    f.create_dataset("trace", data=df.to_records(index=False))
    f.create_dataset("stats", data=stats.to_records(index=False))

In [ ]:
pm.summary(trace)

In [ ]:
median_period = np.median(trace["period"], axis=0)
samples = np.array(trace["period"])
samples[:, ~single_transit] = (
    24 * 60 * (samples[:, ~single_transit] - median_period[None, ~single_transit])
)
samples[:, single_transit] = np.log10(samples[:, single_transit])
labels = [
    f"$\log_{{10}} P_{n + 1} / \mathrm{{day}}$"
    if single_transit[n]
    else f"$\Delta P_{n + 1}$ [min]"
    for n in range(num_toi)
]

fig = corner.corner(samples, labels=labels)
for n, ax in enumerate(np.diag(np.array(fig.axes).reshape(num_toi, num_toi))):
    if single_transit[n]:
        continue
    ax.set_title(f"$P_\mathrm{{ref}} = {median_period[n]:.6f}$", fontsize=14)

In [ ]:
samples = np.concatenate(
    (trace["ror"], trace["b"], trace["transit_duration"] * 24,), axis=1
)
labels = [f"$R_{n + 1} / R_S$" for n in range(num_toi)]
labels += [f"$b_{n + 1}$" for n in range(num_toi)]
labels += [f"$\\tau_{n + 1}$ [hr]" for n in range(num_toi)]

_ = corner.corner(samples, labels=labels)

In [ ]:
star = Catalogs.query_object(f"TIC {tic}", catalog="TIC", radius=0.001)
tic_rho_star = float(star["rho"]), float(star["e_rho"])
if np.all(np.isfinite(tic_rho_star)):

    print("rho_star = {0} ± {1}".format(*tic_rho_star))

    # Extract the implied density from the fit
    rho_circ = np.repeat(trace["rho_circ"], 500, axis=0)
    period = np.repeat(trace["period"], 500, axis=0)

    # Sample eccentricity and omega uniformly
    ecc = np.random.uniform(0, 1, len(rho_circ))
    omega = np.random.uniform(-np.pi, np.pi, len(rho_circ))

    # Compute the "g" parameter from Dawson & Johnson and what true
    # density that implies
    g = (1 + ecc * np.sin(omega)) / np.sqrt(1 - ecc ** 2)
    rho = rho_circ / g[:, None] ** 3

    # Re-weight these samples to get weighted posterior samples
    log_weights = -0.5 * ((rho - tic_rho_star[0]) / tic_rho_star[1]) ** 2

    for n in range(num_toi):
        weights = np.exp(log_weights[:, n] - np.max(log_weights[:, n]))
        if single_transit[n]:
            samples = np.vstack((ecc, omega, np.log10(period[:, n]))).T
            labels = [
                f"$e_{n + 1}$",
                f"$\omega_{n + 1}$",
                f"$\log_{{10}} P_{n + 1} / \mathrm{{day}}$",
            ]
        else:
            samples = np.vstack((ecc, omega)).T
            labels = [f"$e_{n + 1}$", f"$\omega_{n + 1}$"]

        fig = corner.corner(
            samples, weights=weights, plot_datapoints=False, labels=labels,
        )
        fig.suptitle(f"TOI {toi_num}.{n + 1:02d}")

else:
    print("The TIC has no measured density")